In [146]:
%pip install gigachain_community

Note: you may need to restart the kernel to use updated packages.


In [147]:
from langchain_community.chat_models.gigachat import GigaChat

In [ ]:
from dotenv import load_dotenv

In [ ]:
import os

In [ ]:
load_dotenv()

In [ ]:
credentials = os.getenv("GIGACHAT_CREDENTIALS")

In [149]:
def load_user_prompt(file_path): # функция загрузки user_prompt из txt файла
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

In [150]:
SYSTEM_PROMPT = """
ВЫВОД ДОЛЖЕН НАЧИНАТЬСЯ СТРОГО С {"scenes": [ И НИЧЕГО ПЕРЕД ЭТИМ!
Роль: Профессиональный геймдизайнер RPG, специализирующийся на ветвящихся сюжетах и атмосферных мирах.

Требования к генерации:
1. Формат:
- Строго валидный JSON
- Только объект с массивом "scenes"
- Никаких посторонних символов/комментариев

2. Входные данные (заменять фигурными скобками):
- Жанр: {жанр}
- Герой: {герой} 
- Цель: {цель}

3. Структура квеста:
- 5-10 сцен
- Минимум 1 развилка с альтернативной веткой (3+ сцены)
- Все сцены кроме финальных: 2 или 3 выбора
- Финальные сцены: choices = []

4. Технические требования:
- Все scene_id: lowercase_underscore
- Длина text в сцене: 100-500 символов
- Длина text в выборе: 20-100 символов
- Все next_scene должны существовать
- Недопустимы циклы в сценах

5. Качественные критерии:
- Яркая передача атмосферы жанра
- Логичные последствия выборов
- Минимум 1 неочевидный выбор
- Реалистичные NPC с динамичными реакциями
- Соответствие структуре: завязка->конфликт->развязка

Процесс генерации:
1. Создать outline с scene_id
2. Проверить ветвление
3. Наполнить контентом
4. Валидировать JSON
5. Убедиться в выполнении всех требований

Запрещено:
- Нарушать формат JSON
- Создавать невалидные ссылки между сценами
- Использовать неподходящий контент
- Добавлять пояснения к выводу

Шаблон для заполнения:
{
  "scenes": [
    {
      "scene_id": "start",
      "text": "Вы - {герой} в мире {жанр}. {Атмосферное описание}. Ваша цель: {цель}.",
      "choices": [
        {
          "text": "{Логичный выбор 1}",
          "next_scene": "scene_1"
        },
        {
          "text": "{Альтернативный выбор 2}",
          "next_scene": "scene_2"
        }
      ]
    }
    // Дополнить 4-9 сценами
  ]
}

Жесткие технические требования:
1. Вывод ТОЛЬКО в формате JSON без каких-либо посторонних символов
2. Начинаться должен строго с: {"scenes": [
3. Заканчиваться строго на: ]}
4. Никаких комментариев, пояснений или оберток

ВАЖНО: вывод должен быть в чистом JSON, без пояснений или комментариев. Поле scenes обязательно должно быть
"""

In [151]:
from httpx import ReadTimeout

In [152]:
import json

In [153]:
# функция для генерации квеста
# Загружаем user_prompt
def generate_rpg_quest(user_prompt_txt_path, credentials):
    user_prompt = load_user_prompt(user_prompt_txt_path)

    try:
        giga = GigaChat(
            credentials=credentials,
            verify_ssl_certs=False,
            timeout=360,
            model="GigaChat-2-Max"
        )

        full_prompt = f"""{SYSTEM_PROMPT}\n\nВходные данные:\n{user_prompt}\n\nВывод только в JSON!Требования к выводу:
    1. ТОЛЬКО JSON без каких-либо других текстов
    2. Обязательное наличие поля "scenes"
    3. Строго соответствовать шаблону
    """
        response = giga.invoke(full_prompt)

        response_text = response.content

        

        # Извлекаем чистый JSON
        start = response_text.find('{"scenes": [')
        if start < 0:
            raise ValueError("Ответ не содержит валидный JSON")
        end = response_text.rfind(']}') + 2
        json_str = response_text[start:end]

        return json.loads(json_str)
        

    except ReadTimeout:
        print("Ошибка: превышено время ожидания ответа от GigaChat.")
        return {"error": "timeout"}
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return {"error": str(e)}


In [154]:
# if __name__ == "__main__":
#     # Путь к файлу с user_prompt (например, "input.txt")
#     USER_PROMPT_PATH = "input/example-3.txt"  # Замените на ваш путь

#     # Авторизационный ключ GigaChat
#     CREDENTIALS = credentials  # Замените на реальный ключ

#     # Генерация квеста
#     quest = generate_rpg_quest(USER_PROMPT_PATH, CREDENTIALS)
    
#     if not isinstance(quest, dict) or "scenes" not in quest:
#         print("ОШИБКА: Сгенерированный квест не соответствует формату")
#         print("Получено:", quest)
            
#     else:
#         print("Получено:", quest)
#         with open("text_output/example-3.txt", "w", encoding="utf-8") as f:
#             # Записываем как красиво форматированный JSON

#             json.dump(quest, f, indent=2, ensure_ascii=False)

        
#     print("Квест успешно сгенерирован и сохранён в text_output/example-3.txt!")

In [ ]:
import subprocess
import os
from time import sleep

def generate_quest_with_validation(quest_name, credentials, user_prompt_path, max_retries=3):
    """
    Генерирует и ибрабатывает квест через process.py, который:
    1. Читает text_output/{quest_name}.txt
    2. Конвертирует в JSON
    3. Сохраняет в generated_quests/{quest_name}.json
    4. Выполняет валидацию

    При ошибках в stderr перезапускает генерацию квеста (до max_retries раз)
    """
    print(f"Генерируем квест: {quest_name}")
    input_file = f"text_output/{quest_name}.txt"
    quest = generate_rpg_quest(user_prompt_txt_path=user_prompt_path, credentials=credentials)
    with open(input_file, "w") as f:
        f.write(json.dumps(quest))
    print(f"Квест сохранен в {input_file}")
    retry_count = 0

    while retry_count < max_retries:
        try:
            # Проверяем существование входного файла
            if not os.path.exists(input_file):
                print(f"Ошибка: файл {input_file} не найден")
                return False

            print(f"Обрабатываем квест: {quest_name} (попытка {retry_count + 1}/{max_retries})")
            print(f"Входной файл: {input_file}")

            # Запускаем process.py
            result = subprocess.run([
                'python3', 'process.py', f"text_output/{quest_name}.txt", "-o", f"generated_quests/{quest_name}.json"
            ],
            capture_output=True,
            text=True
            )

            # Выводим результат
            if result.stdout:
                print("Вывод process.py:")
                print(result.stdout)

            if result.stderr:
                print("Ошибки:")
                print(result.stderr)

                # Если есть ошибки и мы можем перегенерировать квест
                if credentials is not None and user_prompt_path is not None:
                    retry_count += 1
                    if retry_count < max_retries:
                        print(f"\n⚠️ Обнаружены ошибки, перегенерируем квест (попытка {retry_count + 1}/{max_retries})")

                        quest = generate_rpg_quest(user_prompt_path, credentials)

                        # Сохраняем новый вариант
                        with open(input_file, "w", encoding="utf-8") as f:
                            f.write(quest)

                        print("Квест перегенерирован, повторяем обработку...")
                        sleep(1)
                        continue

            if result.returncode == 0:
                print(f"✅ Квест {quest_name} успешно обработан и валидирован!")

                # Проверяем, что файл создался
                output_file = f"generated_quests/{quest_name}.json"
                if os.path.exists(output_file):
                    print(f"✅ Файл {output_file} создан")
                else:
                    print(f"⚠️ Файл {output_file} не найден")

                return True
            else:
                print(f"❌ Ошибка при обработке квеста {quest_name}")
                return False

        except Exception as e:
            print(f"Критическая ошибка: {e}")
            return False

    print(f"❌ Достигнуто максимальное количество попыток ({max_retries})")
    return False

# Обрабатываем example-3
if __name__ == "__main__":
    quest_name = "example-3"
    max_retries = 3  # Максимальное количество попыток перегенерации
    credentials = os.getenv("GIGACHAT_CREDENTIALS")  # Ваши учетные данные GigaChat
    user_prompt_path = "input/example-3.txt"  # Путь к файлу с промптом

    print(f"Запускаем генерацию квеста: {quest_name}")
    print("=" * 50)

    success = generate_quest_with_validation(
        quest_name=quest_name,
        max_retries=max_retries,
        credentials=credentials,
        user_prompt_path=user_prompt_path
    )
    
    if success:
        print("\n🎉 Обработка завершена успешно!")
        print(f"Квест {quest_name} готов к использованию в generated_quests/{quest_name}.json")
    else:
        print("\n❌ Обработка завершилась с ошибками")
        print("Проверьте логи выше для диагностики проблем")


Запускаем генерацию квеста: example-3
Генерируем квест: example-3
Квест сохранен в text_output/example-3.txt
Обрабатываем квест: example-3 (попытка 1/3)
Входной файл: text_output/example-3.txt
Вывод process.py:
Результат валидации: Все проверки пройдены успешно
JSON файл сохранен: generated_quests/example-3.json

✅ Квест example-3 успешно обработан и валидирован!
✅ Файл generated_quests/example-3.json создан

🎉 Обработка завершена успешно!
Квест example-3 готов к использованию в generated_quests/example-3.json
